# Урок 2. Парсинг HTML. BeautifulSoup, MongoDB

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайтов Superjob и HH. 

Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). 

Получившийся список должен содержать в себе минимум:

* Наименование вакансии.
* Предлагаемую зарплату (отдельно минимальную, максимальную и валюту).
* Ссылку на саму вакансию.
* Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). 

Структура должна быть одинаковая для вакансий с обоих сайтов. 

Общий результат можно вывести с помощью dataFrame через pandas.

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import json
import re
from pprint import pprint

In [2]:
#user_agent
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36'}

#параметры
occupation = 'Python'

## hh.ru

In [3]:
#pages = 5
main_link = 'https://hh.ru'

#запрос данных
response = requests.get(f'{main_link}/search/vacancy?clusters=true&area=1&enable_snippets=true&salary=&st=searchVacancy&text={occupation}', headers=headers)

#преобразование в DOM
soup = bs(response.text, 'lxml')

#основа для списка
hh_vacancy_list = []

#кнопка "Дальше"
pages = soup.find_all('a', {'class': 'bloko-button HH-Pager-Control'})
print(int(pages[-1].text))

#цикл для сбора

if response.ok:
    for page in range(int(pages[-1].text)):
        hh_vacancies_block = soup.find_all('div', {'class': 'vacancy-serp'})
        hh_vacancies_list = hh_vacancies_block[0].find_all('div', {'data-qa': 'vacancy-serp__vacancy vacancy-serp__vacancy_premium'}) + \
                  hh_vacancies_block[0].find_all('div', {'data-qa': 'vacancy-serp__vacancy'})

        for vacancy in hh_vacancies_list: #цикл для сборки словаря
            vacancy_data = {}
            vacancy_data['name'] = vacancy.find_all('a', {'class': 'bloko-link HH-LinkModifier HH-VacancyActivityAnalytics-Vacancy'})[0].getText()
            #'bloko-link HH-LinkModifier'
            vacancy_data['link'] = vacancy.find_all('a', {'class': 'bloko-link HH-LinkModifier HH-VacancyActivityAnalytics-Vacancy'})[0]['href']
            #'bloko-link HH-LinkModifier'
            vacancy_data['employer'] = vacancy.find_all('div', {'class': 'vacancy-serp-item__meta-info-company'})[0].getText()

            try:
                salary = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'}).text
            except AttributeError:
                salary = None

            if salary is None:
                salary_min = None
                salary_max = None
                salary_curr = None
            else:
                salary = salary.replace('\xa0', '')
                salary = salary.replace(' ', '-')
                salary = salary.split('-')
                if salary[0] == 'от':
                    salary_min = int(salary[1])
                    salary_max = None
                elif salary[0] == 'до':
                    salary_min = None
                    salary_max = int(salary[1])
                else:
                    salary_min = int(salary[0])
                    salary_max = int(salary[1])
                salary_curr = salary[2]

            vacancy_data['salary_min'] = salary_min
            vacancy_data['salary_max'] = salary_max
            vacancy_data['salary_currency'] = salary_curr

            hh_vacancy_list.append(vacancy_data)

#сохраняем датафрейм и json
df = pd.DataFrame(hh_vacancy_list)
print(df)
df.to_csv(f'hh_{occupation}.csv', encoding='utf-8')
with open(f'hh_{occupation}.json', 'w', encoding='utf-8') as outfile:
    json.dump(hh_vacancy_list, outfile)

40
                                        name  \
0     Quantitative developer (HFT developer)   
1                            DevOps Engineer   
2        Senior Backend-разработчик (Python)   
3                         Python разработчик   
4                         Python разработчик   
...                                      ...   
1995  Ведущий специалист по Machine learning   
1996                  Data Engineer (Python)   
1997                    Разработчик (Python)   
1998          Python Программист разработчик   
1999               Teamlead Backend (Python)   

                                             link  \
0     https://hh.ru/vacancy/43005025?query=Python   
1     https://hh.ru/vacancy/42301915?query=Python   
2     https://hh.ru/vacancy/41649102?query=Python   
3     https://hh.ru/vacancy/42855358?query=Python   
4     https://hh.ru/vacancy/42766512?query=Python   
...                                           ...   
1995  https://hh.ru/vacancy/42202946?query=Python

## superjob.ru

In [4]:
#pages = 5
main_link = 'https://superjob.ru'

sj_params = {'pages': 1,
             'keywords': occupation,
             'noGeo': 1}

#запрос данных
response = requests.get(main_link + '/vacancy/search/', headers=headers, params=sj_params)

#преобразование в DOM
soup = bs(response.text, 'lxml')

#основа для списка
sj_vacancy_list = []

#цикл для сбора
if response.ok:
    sj_vacancies_block = soup.find('div', {'class': 'iJCa5'})
    sj_vacancies_list = sj_vacancies_block.find_all('div', {'class': 'LvoDO'})
    print(len(sj_vacancies_list))

    for vacancy in sj_vacancies_list: #цикл для сборки словаря
        vacancy_data = {}
        vacancy_data['name'] = vacancy.find('a', {'class': 'icMQ_'}).getText()
        vacancy_data['link'] = main_link + vacancy.find('a', {'class': 'icMQ_'})['href']

        a = vacancy.find('span', {'class': 'PlM3e'}).getText()
        if a == 'По договорённости':
            a = None
        else:
            a = re.split('[\xa0 -]', a)
            a = [i for i in a if i != '—']

            if len(a) == 4 and a[0] == 'от':
                vacancy_data['min_salary'] = int(a[1] + a[2])
            if len(a) == 4 and a[0] == 'до':
                vacancy_data['max_salary'] = int(a[1] + a[2])
            if len(a) > 4:
                vacancy_data['min_salary'] = int(a[0] + a[1])
                vacancy_data['max_salary'] = int(a[2] + a[3])

            vacancy_data['currency'] = a[-1]

        b = vacancy.find('span', {'class': 'clLH5'}).find_next_sibling().getText()
        vacancy_data['city'] = b.split(',')[0]
        vacancy_data['main_link'] = main_link
        vacancy_data['vacancy_id'] = str(''.join(re.findall('[0-9]+', vacancy_data['link'])))

        sj_vacancy_list.append(vacancy_data)

#сохраняем датафрейм и json
df = pd.DataFrame(sj_vacancy_list)
print(df)
df.to_csv(f'sj_{occupation}.csv', encoding='utf-8')
with open(f'sj_{occupation}.json', 'w', encoding='utf-8') as outfile:
    json.dump(sj_vacancy_list, outfile)

20
                                                 name  \
0                      Программист-разработчик Python   
1           Инженер-программист Python (data science)   
2       Automation QA Engineer / Тестировщик (Python)   
3                          BackEnd Python-разработчик   
4                          Backend-разработчик Python   
5           Аналитик базы данных / Разработчик Python   
6                           Senior Python разработчик   
7   Ведущий программист (Python-разработчик (pre-m...   
8                                  Программист Python   
9   Python developer / Python разработчик / Програ...   
10  Продуктовый аналитик (Команда цифровых продукт...   
11                                    DevOps Engineer   
12                                Начальник отдела ИТ   
13              Системный администратор Windows Linux   
14  Инженер-проектировщик ИТ и слаботочных сетей (...   
15                                           Аналитик   
16                          